In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd

/content


In [3]:
import os

path = '/content/drive/MyDrive/EVA6/s7_depthwise_cifar10/'

In [4]:
% cd $path

/content/drive/MyDrive/EVA6/s7_depthwise_cifar10


In [5]:
WORKING_DIR="ComputerVision"

In [6]:
!if [ -d "$WORKING_DIR" ]; then rm -Rf $WORKING_DIR; fi

In [7]:
!git clone https://github.com/joyjeni/ComputerVision/

Cloning into 'ComputerVision'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 292 (delta 115), reused 111 (delta 38), pack-reused 0
Receiving objects: 100% (292/292), 776.00 KiB | 4.46 MiB/s, done.
Resolving deltas: 100% (115/115), done.


In [8]:
%cd ComputerVision/s7_cifar10_Depthwise_Separable_Conv/Pybeam

/content/drive/MyDrive/EVA6/s7_depthwise_cifar10/ComputerVision/s7_cifar10_Depthwise_Separable_Conv/Pybeam


#### Run the CIFAR10 Model for 50 Epochs

In [9]:
!python main.py --config=experiments/cifar10_config.yml --device=0

[ 2021-06-19 00:26:01,383 - beam.__main__ ] INFO: Training: {'name': 'CIFAR10_MyNet', 'save_dir': 'saved/', 'seed': 1, 'target_device': 0, 'arch': {'type': 'CIFAR10Model', 'args': {}}, 'augmentation': {'type': 'CIFAR10Transforms', 'args': {}}, 'data_loader': {'type': 'CIFAR10DataLoader', 'args': {'batch_size': 64, 'data_dir': 'data/', 'nworkers': 4, 'shuffle': True}}, 'loss': 'nll_loss', 'optimizer': {'type': 'SGD', 'args': {'lr': 0.008, 'momentum': 0.95}}, 'training': {'epochs': 50}}
[ 2021-06-19 00:26:01,386 - beam.__main__ ] INFO: Building: beam.model.model.CIFAR10Model
[ 2021-06-19 00:26:01,420 - beam.__main__ ] INFO: Using device 0 of available devices [0]
[ 2021-06-19 00:26:04,601 - beam.__main__ ] INFO: Building: torch.optim.SGD
[ 2021-06-19 00:26:04,601 - beam.__main__ ] INFO: Building: beam.data_loader.augmentation.CIFAR10Transforms
[ 2021-06-19 00:26:04,601 - beam.__main__ ] INFO: Building: beam.data_loader.data_loaders.CIFAR10DataLoader
170499072it [00:03, 43164608.10it/s]  

In [10]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [11]:
class CIFAR10Model(nn.Module):

    def __init__(self, dropout_value=0.25):

        self.dropout_value = dropout_value  # dropout value

        super(CIFAR10Model, self).__init__()

        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32,
                      kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        )  # output_size = 32

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64,
                      kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        )  # output_size = 32

        # TRANSITION BLOCK 1
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=32,
                      kernel_size=(1, 1), padding=0, bias=False,stride=2),
        )  # output_size = 32
      

        # CONVOLUTION BLOCK 2
        # DEPTHWISE CONVOLUTION AND POINTWISE CONVOLUTION
        self.depthwise1 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64,
                      kernel_size=(3, 3), padding=0, groups=32, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        )  # output_size = 16
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128,
                      kernel_size=(1, 1), padding=0, bias=False,stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        )  # output_size = 16

       
        # CONVOLUTION BLOCK 3
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128,
                      kernel_size=(3, 3), padding=4, dilation=2, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        )  # output_size = 11
        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128,
                      kernel_size=(3, 3), padding=1, bias=False,stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        )  # output_size = 11

        # OUTPUT BLOCK
        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=5)
        )  # output_size = 1

        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128,
                      kernel_size=(1, 1), padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Dropout(self.dropout_value)
        )

        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=10,
                      kernel_size=(1, 1), padding=0, bias=False),
        )

        self.dropout = nn.Dropout(self.dropout_value)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)

        x = self.depthwise1(x)
        x = self.convblock4(x)

        x = self.convblock5(x)
        x = self.convblock6(x)

        x = self.gap(x)
        x = self.convblock7(x)
        x = self.convblock8(x)

        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [12]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = CIFAR10Model().to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
           Dropout-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          18,432
       BatchNorm2d-6           [-1, 64, 32, 32]             128
              ReLU-7           [-1, 64, 32, 32]               0
           Dropout-8           [-1, 64, 32, 32]               0
            Conv2d-9           [-1, 32, 16, 16]           2,048
           Conv2d-10           [-1, 64, 14, 14]             576
      BatchNorm2d-11           [-1, 64, 14, 14]             128
             ReLU-12           [-1, 64, 14, 14]               0
          Dropout-13           [-1, 64, 14, 14]               0
           Conv2d-14            [-